In [0]:
def f(x,y):
          num = x*y
          return num
  
def g(x,y):
          num = (x+2)*(y+3.5)-200
          return num



In [214]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    #res = x**2 + y**2 + z**2
    res = f(x,y)+z*g(x,y)
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 5
    child_num = 10

    space = OrderedDict((
        ('x', (-15., 15.)),
        ('y', (-15., 15.)),
        ('z', (0., 2.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval -7.574132156367944, 12.067579844340266, 1.0770539120498435: -400.27438517445745
Eval -4.4818222517534885, -0.40998492582877333, 1.9760691302138713: -408.5305601324078
Eval 8.165498078808, 10.034713215343466, 1.9813533693429948: -41.72357587075693
Eval -5.371538578573304, 5.914308158064554, 0.43649298917617374: -132.92212666637872
Eval -0.17063915964848597, -10.786018926487499, 1.5751147703181456: -334.1767599910506
Eval 0.3235863074562726, 9.828586547752874, 0.00017526116895494326: 3.150771654354964
Eval 7.648359500998421, 13.371229680133343, 0.4839536069956525: 84.2550678881968
Eval -13.431001962842592, -2.0354702121796624, 1.3168281957443073: -258.0723120232845
Eval 5.66908107134352, -10.04852806847109, 1.0097555291654772: -309.6281530656739
Eval 8.53758350642089, -4.978128671247628, 0.8903272460375156: -234.4342904297439
Best [-4.4818222517534885, -0.40998492582877333, 1.9760691302138713]

Eval 9.430590350354326, 6.6625974869193545, 2.0: -104.83879489453236
Eval 8.0720702915165